# Challenge - Jigsaw Unintended Bias in Toxicity Classification

![](https://images.unsplash.com/photo-1581686051110-30d45d3e1fd8?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1050&q=80)

⚠️ **Transformers are huge models and in order to train / fine-tune such models you need a GPU. Therefore you will need to run this notebook on Colab.**

⚠️ **Please upload this notebook on Colab.**

⚠️ **Do no forget to use GPU as hardware accelerator. To do that go to `Edit -> Notebook Settings` and select GPU at the field `Hardware Accelerator`**

Let's select the Tensorflow 2.0 version to run this Colab notebook:

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

## Introduction

This challenge is based on an interesting Kaggle Competition: the [Jigsaw Toxic Comment Clasffication Challenge](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge).


The main goal of the exercise is to predict toxicity of comments.

You are provided with a large number of Wikipedia comments which have been labeled by human raters for toxic behavior. There are 6 types of toxicity:
- toxic
- severe_toxic
- obscene
- threat
- insult
- identity_hate

Each comment has a label for each one of the categories. A label equals to 1 means that comment is considered toxic in that category.

Given the memory needs for training / fine-tuning a Transformer, you will be using only one of these categories as target. Feel free to chose one.

## Downloading Data

In order to use the data available for this competion by kaggle on Colab you need to follow the steps below:
1. Make sure you have a Kaggle account (you can subscribe using your Google Account for example)
2. Go to `My Account` 
3. At Section API click on `Create New API Token`. A `json` file will be automatically downloaded
4. Open the `json` file and substitute the username and key in the cell below by the data provided on the file 
5. Run the cell below

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "your_username"
os.environ['KAGGLE_KEY'] = "your_kaggle_key"

!pip install -q kaggle
!pip install -q kaggle-cli

!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge
!unzip train.csv.zip
!unzip test.csv.zip
!unzip test_labels.csv.zip

!pip install transformers

## Loading data

As usual, load the training data using pandas (file `train.csv`)

In [ ]:
# ENTER YOUR CODE HERE

## Preprocessing

Now we are going to prepare the data before training our model.

Choose one category as your target.

Also select the `comment_text` column to a variable `comments` so that we can process it using Transformers tokenizers.

In [ ]:
CATEGORY = # ENTER YOUR CODE HERE

targets = # ENTER YOUR CODE HERE
comments = # ENTER YOUR CODE HERE

Decide a transformer model you want to use (eg: DistilBert, Bert, etc) and import its correspondent tokenizer.

Use the method `.from_pretrained` to load the tokenizer.

You can refer to the lecture to see how you do it.

You can see the list of pretrained transformers models available here: https://huggingface.co/transformers/pretrained_models.html

> *As an advice, for this exercise, prefer to use a version of DistilBert, as it is smaller and takes less time to train than the majority of Transformers*

In [ ]:
from transformers import # ENTER YOUR CODE HERE

tokenizer = # ENTER YOUR CODE HERE

Complete the function `process_comments` below in order to process the text and encode it in a fashion that it can be used be your Transformer model.

As you are padding the sentences, do not forget to store the attention mask, as we saw during the lecture.

In [ ]:
from tqdm import tqdm

def process_comments(tokenizer, comments, max_length):
    input_ids, attention_mask = [], []
    for comment in tqdm(comments):
        
        # ENTER YOUR CODE HERE
        
    return input_ids, attention_mask

Choose a max length for your sentences and generate all input ids and the attention mask:

In [ ]:
MAX_LENGTH = # ENTER YOUR CODE HERE

input_ids, attention_mask = # ENTER YOUR CODE HERE

Using the `train_test_split` method from sci-kit learn, build a training set and a evaluation set. Feel free to choose the size of the evaluation set:

In [ ]:
# ENTER YOUR CODE HERE

Using the method `tf.data.Dataset.from_tensor_slices`, build the training and validation `tf.Datasets`.

You will need to feed this method with the inputs ids, the attention mask and the train targets.
> *Hint: use a dictionnaire to feed both input ids and attention mask*

In [ ]:
train_dataset = # ENTER YOUR CODE HERE
val_dataset = # ENTER YOUR CODE HERE

Now shuffle and batch the datasets (you can refer to [Tensorflow documentation](https://www.tensorflow.org/tutorials/load_data/numpy)):

In [ ]:
BATCH_SIZE = # ENTER YOUR CODE HERE
SHUFFLE_BUFFER_SIZE = # ENTER YOUR CODE HERE

train_dataset = # ENTER YOUR CODE HERE
val_dataset = # ENTER YOUR CODE HERE

## Modelling and training

Now that you have your data well prepared, import and build a pretrained Transformer model from the `transformers` library.

Please be sure that you are using a model for `SequenceClassification` as our task here is basically the creation of a classifier that detects toxicities in sentences.

In [ ]:
from transformers import # ENTER YOUR CODE HERE

model = # ENTER YOUR CODE HERE

Now, as usual, define the optimizer, the loss and the metrics that will be used during training and compile the model.

In [ ]:
# ENTER YOUR CODE HERE

Now let's train our transformer!

> *Obs: It will take about 30min for each epoch during the training, therefore you can launch the training for just very few epochs and go take a coffee or you can start taking a look at the challenge 02.*

> *Optional: Use an `early_stopping` callback to avoid overfitting during training*

In [ ]:
# ENTER YOUR CODE HERE

## Computing predictions

Now that you trained the your model, let's load the test samples and evaluate the model on it.

Load the test data and test labels.

In [ ]:
test_data = # ENTER YOUR CODE HERE

In [ ]:
test_labels = # ENTER YOUR CODE HERE

Process the test comments as we proceeded with the training data and select the category you used to train your model.

**Please be sure you eliminate the rows with labels == -1, these rows should not be used for evaluation.**

In [ ]:
# Select the targets and comments with labels != -1
test_targets = # ENTER YOUR CODE HERE
test_comments = # ENTER YOUR CODE HERE

# Process the comments
test_inputs, test_mask = # ENTER YOUR CODE HERE

Now compute the predictions from your model:

In [ ]:
predictions = # ENTER YOUR CODE HERE

You can see that the predictions (the Transformer's output) are a 2-dim vector with values from  `-inf` to `inf`. 

In [ ]:
predictions

These values correspond to the logits that are outputted from the model.

In order to compute the probability of label == 1 we should use the softmax function:

In [ ]:
from scipy.special import softmax

probas = softmax(predictions, axis=1)

probas

We can notice that the classes are very unbalanced (there are much less toxic comments than non toxic comments):

In [ ]:
n_toxic = sum(probas[:,1] > 0.5)
n_non_toxic = sum(probas[:,0] > 0.5)

print("Proportion of toxic comments: {0:.2%}".format(n_toxic/(n_toxic + n_non_toxic)))

Therefore, in order to evaluate properly our model, we should be using a metric that takes this unbalaceness into account. The AUC score is one of these metrics, and it is the one used to evaluate the performance of the competitors in the original Kaggle competition.

Import the `roc_auc_score` function for the scikit-learn library and use it to evaluate your predictions.

Please be sure you use the probability of label == 1 computed by your model as input to the `roc_auc_score` function.

In [ ]:
# ENTER YOUR CODE HERE

## (Optional) Do the same for other classes

As a optional exercise, you can repeat the same steps and train other transformer models for the other 5 categories.

It will take some time, but after doing it, you can submit your predictions to the official competition and check your potential ranking! 